In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import numpy as np

# Load trained scaler, PCA, and model
scaler = joblib.load("models/scaler.pkl")         # trained on 42 features
pca = joblib.load("models/pca.pkl")               # PCA trained on scaled 42 features
model = joblib.load("models/kmeans_model.pkl")    # trained on 2 PCA components

# Cluster labels
cluster_labels = {
    0: "Balanced Middle-Aged",
    1: "Younger Low-Balance",
    2: "High-Balance Low Campaign",
    3: "Younger Lower-Balance"
}

class KMeansGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Customer Cluster Predictor")
        self.geometry("980x780")
        self.configure(bg="#121212")

        self.inputs = {}
        self._define_fields()
        self._build_ui()

    def _define_fields(self):
        self.fields = {
            "Age": [str(i) for i in range(18, 71, 5)],
            "Balance": [str(i) for i in range(0, 10001, 500)],
            "Day": [str(i) for i in range(1, 32)],
            "Duration": [str(i) for i in range(30, 601, 30)],
            "Campaign": [str(i) for i in range(1, 11)],
            "Days Passed": [str(i) for i in [-1, 3, 5, 10, 15, 999]],
            "Previous": [str(i) for i in range(0, 6)],
            "Job": ["admin.", "blue-collar", "entrepreneur", "housemaid", "management", "retired",
                    "self-employed", "services", "student", "technician", "unemployed", "unknown"],
            "Marital": ["divorced", "married", "single"],
            "Education": ["primary", "secondary", "tertiary", "unknown"],
            "Has Default": ["Yes", "No"],
            "Has Housing Loan": ["Yes", "No"],
            "Has Personal Loan": ["Yes", "No"],
            "Contact Method": ["cellular", "telephone", "unknown"],
            "Last Contact Month": ["aug", "dec", "feb", "jan", "jul", "jun", "mar", "may", "nov", "oct", "sep"],
            "Previous Outcome": ["success", "other", "unknown"]
        }

    def _build_ui(self):
        tk.Label(self, text="Customer Segmentation Predictor", font=("Segoe UI", 20, "bold"),
                 bg="#121212", fg="#00cec9").pack(pady=20)

        form_frame = tk.Frame(self, bg="#121212")
        form_frame.pack()

        for idx, (label_text, options) in enumerate(self.fields.items()):
            row, col = divmod(idx, 2)
            label = tk.Label(form_frame, text=label_text, font=("Segoe UI", 11), bg="#121212", fg="white")
            label.grid(row=row, column=col * 2, padx=10, pady=6, sticky="e")
            cb = ttk.Combobox(form_frame, values=options, state="readonly", width=25)
            cb.set("")
            cb.grid(row=row, column=col * 2 + 1, padx=10, pady=6, sticky="w")
            self.inputs[label_text] = cb

        self.predict_btn = tk.Button(self, text="Predict Segment", font=("Segoe UI", 12, "bold"),
                                     bg="#00cec9", fg="black", activebackground="#55efc4",
                                     command=self.predict_cluster, padx=30, pady=10, bd=0, relief="ridge")
        self.predict_btn.pack(pady=25)

        self.result_box = tk.Label(self, text="", bg="#1e1e1e", fg="white",
                                   font=("Segoe UI", 16), width=50, height=3,
                                   bd=2, relief="sunken", wraplength=600, justify="center")
        self.result_box.pack(pady=15)

    def predict_cluster(self):
        self.result_box.config(text="Analyzing...", fg="#dfe6e9")
        self.after(1000, self._do_prediction)

    def _do_prediction(self):
        try:
            # Exact one-hot columns used during training
            job_opts = ["job_blue-collar", "job_entrepreneur", "job_housemaid", "job_management",
                        "job_retired", "job_self-employed", "job_services", "job_student",
                        "job_technician", "job_unemployed", "job_unknown"]
            marital_opts = ["marital_married", "marital_single"]  # 'divorced' was dropped
            education_opts = ["education_secondary", "education_tertiary", "education_unknown"]
            contact_opts = ["contact_telephone", "contact_unknown"]
            month_opts = ["month_aug", "month_dec", "month_feb", "month_jan", "month_jul", "month_jun",
                          "month_mar", "month_may", "month_nov", "month_oct", "month_sep"]
            poutcome_opts = ["poutcome_other", "poutcome_success", "poutcome_unknown"]
            binary_map = {"Yes": 1, "No": 0}

            x = []

            # Numeric fields
            for field in ["Age", "Balance", "Day", "Duration", "Campaign", "Days Passed", "Previous"]:
                val = self.inputs[field].get()
                if val == "":
                    raise ValueError(f"Missing value for {field}")
                x.append(float(val))

            # One-hot encoders
            def encode(selected, options, prefix):
                return [1 if f"{prefix}_{selected}" == col else 0 for col in options]

            x += encode(self.inputs["Job"].get(), job_opts, "job")
            x += encode(self.inputs["Marital"].get(), marital_opts, "marital")
            x += encode(self.inputs["Education"].get(), education_opts, "education")
            x.append(binary_map[self.inputs["Has Default"].get()])       # default_status_yes
            x.append(binary_map[self.inputs["Has Housing Loan"].get()])  # housing_yes
            x.append(binary_map[self.inputs["Has Personal Loan"].get()]) # loan_yes
            x += encode(self.inputs["Contact Method"].get(), contact_opts, "contact")
            x += encode(self.inputs["Last Contact Month"].get(), month_opts, "month")
            x += encode(self.inputs["Previous Outcome"].get(), poutcome_opts, "poutcome")

            if len(x) != 42:
                raise ValueError(f"Expected 42 features, got {len(x)}")

            x_scaled = scaler.transform([x])
            x_pca = pca.transform(x_scaled)
            cluster = model.predict(x_pca)[0]

            label = cluster_labels.get(cluster, f"Cluster {cluster}")
            self.result_box.config(text=f"✅ Predicted Segment: {label}", fg="#00cec9")

        except Exception as e:
            self.result_box.config(text="❌ Prediction Failed", fg="red")
            messagebox.showerror("Error", str(e))

# === Run the app ===
app = KMeansGUI()
app.mainloop()








/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
